In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2017,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath   = os.path.join('../')
print('datapath is set to %s'%datapath)

# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install plotly 
# !pip install cufflinks
#!pip install colorspacious
#!pip install seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile


In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
#if not 'progress' in globals(): progress = list()
#if not 'dataloaded' in progress:
  # Read files
for i in [1]:
    df_EC           = Read_LoobosEddFinal    (years,datapath)
    df_Stor         = Read_LooStor           (years,datapath)
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    df_NEE          = Read_Loobos_halfhourly (years,datapath)
    df_meteo        = Read_Loobos_meteo      (years,datapath)
    df_soil         = Read_Loobos_soil       (years,datapath) 
    df_profile      = Read_Loobos_profile    (years,datapath)
    progress.append('dataloaded')

In [ ]:
# Make filter for GPP orginial data and not gabfilled

# Filter for optimisation Ags model

IPAR = ((df_Comb['GPP_fqc']==0)&(df_meteo['PAR']>5))
IVPD = (df_Comb['VPD'] >= 0.1)

# precipitation filter
precip = df_meteo['P(mast)']
print('precip= ',precip)
# precip in current half hour
I0     = (precip > 0)
print('Io= ',I0)
# precip in previous half hour?
I1 = np.zeros(I0.shape)
I1[1:] = I0[:-1]
print('I1= ',I1)
# precip two half hours ago?
I2 = np.zeros(I0.shape)
I2[2:] = I0[:-2]
print('I2= ',I2)
I = I0 + I1 + I2
# precip three half hours ago?
I3 = np.zeros(I0.shape)
I3[3:] = I0[:-3]
I = I0 + I1 + I2
# precip four half hours ago?
I4 = np.zeros(I0.shape)
I4[4:] = I0[:-4]
I = I0 + I1 + I2
print('I= ',I)
# Which half hours did not have precip in the previous two half hours?
II = (I == 0)

# Filter for CO2 data
t = df_profile.index                                          
time = (t < np.datetime64('2013-05-08')) | (t > np.datetime64('2013-06-01'))
CO2 = (df_profile['CO2level1'] > 300)

df_meteo_CO2 = df_meteo[time][CO2]
df_meteo_Prec= df_meteo_CO2[II]
df_meteo_VPD = df_meteo_Prec[IVPD]
df_meteo_filter = df_meteo_VPD[IPAR]

df_Comb_CO2 = df_Comb[time][CO2]
df_Comb_Prec = df_Comb_CO2[II]
df_Comb_VPD = df_Comb_Prec[IVPD]
df_Comb_filter = df_Comb_VPD[IPAR]


df_profile_CO2 = df_profile[time][CO2]
df_profile_Prec = df_profile_CO2[II]
df_profile_VPD = df_profile_Prec[IVPD]
df_profile_filter = df_profile_VPD[IPAR]

df_EC_CO2 = df_EC[time][CO2]
df_EC_Prec = df_EC_CO2[II]
df_EC_VPD = df_EC_Prec[IVPD]
df_EC_filter = df_EC_VPD[IPAR]


# print(df_meteo_filter, df_Comb_filter, df_profile_filter)

In [ ]:
GPP_f = df_Comb_filter['GPP_f']/1000000 * 44.01 * 1000 # from umolm-2s-1 to molm-2s-1, to gm-2s-1, to mgm-2s-1
print(GPP_f)

In [ ]:
# CODE after optimisation

#Calculate canopy resistance and CO2 assimilation/respiration
def runAgs():
    
            #fluxes using the A-Gs scheme.
        co2_ppm   = df_profile_filter['CO2level1']
        epsi      = 1.   # epsilon
        sigma     = 5.67E-8
        Tair_K    = df_Comb_filter['Tair'] + 273.  # This is the air temperature
        Ts_K      = ((df_meteo_filter['L(o)corr'] / (epsi * sigma))**0.25) # This is the surface temperature, which should be used in the model
        Ts_C      = Ts_K - 273.
        conv_fac  = 101.3 / (8.314 * Tair_K)       # converstion factor, obtained via the ideal gas law. mol / m3
        co2_mgm3  = (co2_ppm * 44.01) * conv_fac   # concentration * conversion factor * molar mass CO2.  mgm3 = ppm * g/mol / mol/m3
        Ts        = Ts_C

        rho_1     = 1.225            # Density of air kg/m3

            # Fixed constants 
        Q10gm     = 2.0              # Parameter to calculate the mesophyll conductance
        Q10am     = 2.0              # Parameter to calculate max primary productivity
        Q10gamma  = 2                # Parameter to calculate the CO2 compensation concentration. (2 in IFS, 1.5 in DALES)

            # Reference temperatures calculation mesophyll conductance:
        T1gm      = 273 - 273        # Converted to degreesC
        T2gm      = 309 - 273        # IFS=309, DALES=301 (default)

            # Reference temperatues calculation max primary productivity:
        T1Am      = 273 - 273        # IFS=281, DALES=286 (C4))   Converted to degrees C
        T2Am      = 313 - 273        # IFS=309, DALES=301 

        nuco2q    = 1.6              # Ratio molecular viscosity water to carbon dioxide
        gmin      = 0.25 / 1000.     # Cuticular (minimum) conductance. NOTE: = g_cu in IFS, with a factor 1000 difference (m/s)
        ad        = 0.07             # Regression coefficient to calculate Cfrac (kpa-1)
        Kx        = 0.7              # Extinction coefficient PAR (mground / mleaf)

            # Maximum quantum use efficiency
        epsilon0  =  0.0144    # Maximum quantum use efficiency. mgCO2 / J PAR. Also named alpha

            # Vegetation specific constants
        gm298_umol    = 0.09                        # obtained from litature: Knauer et al. 2018: Effects of mesophyll conductance .....
        gm298         = gm298_umol / conv_fac       # converted to (mm/s)
        Ammax298      = 2.6                         # CO2 maximal primary productivity
        f0            = 0.89                        # Maximum value Cfrac 
        co2_comp298   = (42 * 44.01) * (1/24.45)    # from ppm to mg/m3. Got value 42 from the Atmospheric boundary layer book

        #LAI trees (m2 m-2)
        LAI           = 2.1                         # Obtained from data measurements in Loobos 2021.

            # Constant molar mass
        constants_M_co2 = 44.01
        constants_M_air = 28.97


            # Calculate the CO2 compensation concentration (IFS eq. 8.92)
            # "The compensation point Γ is defined as the CO2 concentration at which the net CO2 assimilation of a fully lit leaf becomes zero."

        co2_comp = co2_comp298 * Q10gamma ** ((Ts - 25) / 10) # equation 8.92. co2_comp = mg/m3.

            # Calculate the mesophyll conductance (IFS eq. 8.93)
            # "The mesophyll conductance gm describes the transport of CO2 from the substomatal cavities to the mesophyll cells where the carbon is fixed."

        gm       = (gm298 * Q10gm **((Ts -25)/10)) / ((1. + np.exp(0.3*(T1gm - Ts)))*(1. + np.exp(0.3*(Ts - T2gm)))) 
        gm       = gm / 1000. # convert to m/s

            # Calculate CO2 concentration inside the leaf (Ci)
        fmin0    = gmin/nuco2q - (1./9.) * gm

            # Calculate the minimum value of Cfrac
        fmin     = gmin /(gmin +gm) # Formula from IFS
        # fmin    = -fmin0 + ((fmin0 **2) + 4* gmin/nuco2q *gm)**0.5 / (2. *gm) # formula from DALES


        VPD      = df_Comb_filter['VPD']/10     #Our measurement data from Loobos converted to kPa (/10). Ds in Dales

        VPDmax   = (f0 - fmin) /ad   # VPDmax in kPa. Dmax in Dalese

            # Calculate the fraction of the concentration inside the leaf in comparison with the surface of the leaf. 
        cfrac    = f0 * (1 - VPD/VPDmax) + fmin * (VPD/VPDmax) # f in IFS.

            # Absolute CO2 concentration (mg/m3)
        co2_abs  = co2_mgm3 

            # CO2 concentration in leaf (mg/m3)
        ci       = cfrac * (co2_abs - co2_comp) + co2_comp

            # Max gross primary production in high light conditions 
            #  line 439 / formula 8.94. Ammax is in mg/m2/s
        Ammax    = (Ammax298 * Q10am ** ((Ts - 25)/10)) / ((1. + np.exp(0.3*(T1Am - Ts)))*(1. + np.exp(0.3*(Ts - T2Am))))

            # Gross assimilation rate (Am, IFS eq. 8.97). In mg/m2/s
        Am       = Ammax * (1 - np.exp(-(gm *(ci - co2_comp) / Ammax))) 

            # Autotrophic dark respiration (IFS eq. 8.99). In mg/m2/s
        Rdark    = Am / 9
 
            # Photosynthetically active radiation (PAR), Ia
        PAR      = df_meteo_filter['PAR'] * 0.22 # measured Loobos data. Convert from umol m-2 s-1 to Jm-2s-1

            # Calculate e (maximum quantum use efficiency) Also named as alpha. mgCO2 / J PAR
        epsilon  = epsilon0 * (co2_abs - co2_comp)/(co2_abs + 2. * co2_comp) # Formula from DALES

            # calculate the gross primary productivity (mg/m2/s)            
        Ag       = (Am + Rdark) * (1 - np.exp((-epsilon * PAR)/(Am + Rdark))) - Rdark # Formula 8.98

        
            # Calculate upscaling from leaf to canopy: net flow CO2 into the plant (An) [-]   
        tempy    = epsilon * Kx * PAR / (Am + Rdark)

        def E1(x):
            # E1() approximation
                euler = 0.5772156649015329
                G     = np.exp(-euler)
                b     = (2*(1-G)/(G*(2-G)))**0.5
                h_inf = (1-G)*(G**2 - 6*G+12) / (3*G*(2-G)**2*b)
                q     = 20/47*x**(31/26.)**0.5
                h     = 1 / (1+x*x**0.5)
                E1    = np.exp(-x) / (G+(1-G)*np.exp(-x/(1-G))) * np.log(1+G/x-(1-G)/(h+b*x)**2)
                return E1

            # Calculate the net assimilation

                # 1.- calculate upscaling from leaf to canopy: net flow CO2 into the plant  
        E1_first    = E1(tempy * np.exp(-Kx*LAI))
        E1_second   = E1(tempy)
        An_canopy   = (Am + Rdark) * (1 - 1. / (Kx * LAI) * (E1_first - E1_second)) # code from DALES

                # 2.- calculate upscaling from leaf to canopy: CO2 conductance at canopy level
        a1          = 1.0 / (1 - f0)
        Dstar       = VPDmax / (a1 * (f0 - fmin))

        fstr        = 1.     # ranges from 0: values at wilting point, to 1: absence of moisture stress
        gcco2       = LAI * (gmin / nuco2q + a1 * fstr * An_canopy / ((co2_abs - co2_comp) * (1. + VPD / Dstar))) # m/s

                # 3. calculate surface resistance for moisture and carbon dioxide
        rs          = 1. / (1.6 * gcco2)
        rsCO2       = 1. / gcco2         # Surface resistance of CO2 in s/m

                # calculate the ra, aerodynamic resistance
        U           = df_EC_filter['Mea_Windsp']
        U_star      = df_EC_filter['U-star']
        ra          = U / (U_star**2)             # get the ra from the Loobos observations


        # 4.  calculate net flux of CO2 into the plant (An, mg/m2/s)
        An_final    = (co2_abs - ci) / (ra + rsCO2)   # should have as default a minus sign before the formula
        # The assimilation rate (A) is expressed as amount of CO2 assimilated per unit leaf area and time (mol m−2 s−1)


        return(An_final)

In [ ]:
an_final = runAgs()

In [ ]:
an_final

In [ ]:
plt.rcParams["figure.figsize"] = (8,4)

In [ ]:
an_final.loc['2017-01-01 00:00':'2017-12-31 00:00'].resample('12H').mean().plot(title='Assimilation in 2017, resampled to 12 hours',ylabel='Assimilation [mg/m2/s]')

In [ ]:
an_final.loc['2018-01-01 00:00':'2019-12-31 00:00'].resample('12H').mean().plot(title='Assimilation in 2018, resampled to 12 hours',ylabel='Assimilation [mg/m2/s]')

## comparison plots

In [ ]:

fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
fig.subplots_adjust(right=0.75)
plt.rcParams["figure.figsize"] = (8,4)

#construct twin axes for supporting up to 2 lines with different y axes
#twin1 = ax.twinx()

data_An = an_final.loc['2017-05-01 00:00':'2017-12-31 00:00'].resample('12H').mean()
data_GPP = df_Comb['GPP_f'].loc['2017-05-01 00:00':'2017-12-31 00:00'].resample('12H').mean()

#plot values
p1 = ax.plot(data_An.index,data_An.values, color="b", label="A-gs Assimilation")
p2, = ax.plot(data_GPP.index,data_GPP.values, color="r", label="GPP_measured")

    

#ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
#twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])

        
    #twin1.invert_yaxis()
    #set axis labels
ax.set_xlabel("Time")
ax.set_ylabel("Assimilation [mg/m2/s]")
#ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='1M'))

fig.suptitle('Assimilation in 2017, resampled to 12 hours')
    
fig.autofmt_xdate() #allows you to auto-align the x axis labels

ax.legend()

plt.show()

#an_final.loc['2017-01-01 00:00':'2017-12-31 00:00'].resample('12H').mean().plot(title='Assimilation in 2017, resampled to 12 hours',ylabel='Assimilation [mg/m2/s]')

## 2017

In [ ]:

fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
fig.subplots_adjust(right=0.75)
plt.rcParams["figure.figsize"] = (8,4)

#construct twin axes for supporting up to 2 lines with different y axes
#twin1 = ax.twinx()

data_An = an_final.loc['2017-05-01 00:00':'2017-12-31 00:00'].resample('12H').mean()
data_GPP = df_Comb['GPP_f'].loc['2017-05-01 00:00':'2017-12-31 00:00'].resample('12H').mean()

#plot values
p1 = ax.plot(data_An.index,data_An.values, color="b", label="A-gs Assimilation")
#p2, = ax.plot(data_GPP.index,data_GPP.values, color="r", label="GPP_measured")

    

#ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
#twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])

        
    #twin1.invert_yaxis()
    #set axis labels
ax.set_xlabel("Time")
ax.set_ylabel("Assimilation [mg/m2/s]")
#ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='1M'))

fig.suptitle('Assimilation in 2017, resampled to 12 hours')
    
fig.autofmt_xdate() #allows you to auto-align the x axis labels

ax.legend()

plt.show()

#an_final.loc['2017-01-01 00:00':'2017-12-31 00:00'].resample('12H').mean().plot(title='Assimilation in 2017, resampled to 12 hours',ylabel='Assimilation [mg/m2/s]')

In [ ]:

fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
fig.subplots_adjust(right=0.75)
plt.rcParams["figure.figsize"] = (8,4)

#construct twin axes for supporting up to 2 lines with different y axes
#twin1 = ax.twinx()

#data_An = an_final.loc['2017-05-01 00:00':'2017-12-31 00:00'].resample('12H').mean()
data_GPP = df_Comb['GPP_f'].loc['2017-05-01 00:00':'2017-12-31 00:00'].resample('12H').mean()

#plot values
#p1 = ax.plot(data_An.index,data_An.values, color="b", label="A-gs Assimilation")
p1, = ax.plot(data_GPP.index,data_GPP.values, color="r", label="GPP_measured")

    

#ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
#twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])

        
    #twin1.invert_yaxis()
    #set axis labels
ax.set_xlabel("Time")
ax.set_ylabel("Assimilation [mg/m2/s]")
#ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='1M'))

fig.suptitle('Assimilation in 2017, resampled to 12 hours')
    
fig.autofmt_xdate() #allows you to auto-align the x axis labels

ax.legend()

plt.show()

#an_final.loc['2017-01-01 00:00':'2017-12-31 00:00'].resample('12H').mean().plot(title='Assimilation in 2017, resampled to 12 hours',ylabel='Assimilation [mg/m2/s]')

## 2018

In [ ]:

fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
fig.subplots_adjust(right=0.75)
plt.rcParams["figure.figsize"] = (8,4)

#construct twin axes for supporting up to 2 lines with different y axes
#twin1 = ax.twinx()

data_An = an_final.loc['2018-05-01 00:00':'2018-12-31 00:00'].resample('12H').mean()
data_GPP = df_Comb['GPP_f'].loc['2018-05-01 00:00':'2018-12-31 00:00'].resample('12H').mean()

#plot values
p1 = ax.plot(data_An.index,data_An.values, color="b", label="A-gs Assimilation")
#p2, = ax.plot(data_GPP.index,data_GPP.values, color="r", label="GPP_measured")

    

#ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
#twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])

        
    #twin1.invert_yaxis()
    #set axis labels
ax.set_xlabel("Time")
ax.set_ylabel("Assimilation [mg/m2/s]")
#ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='1M'))

fig.suptitle('Assimilation in 2017, resampled to 12 hours')
    
fig.autofmt_xdate() #allows you to auto-align the x axis labels

ax.legend()

plt.show()

#an_final.loc['2017-01-01 00:00':'2017-12-31 00:00'].resample('12H').mean().plot(title='Assimilation in 2017, resampled to 12 hours',ylabel='Assimilation [mg/m2/s]')

In [ ]:

fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
fig.subplots_adjust(right=0.75)
plt.rcParams["figure.figsize"] = (8,4)

#construct twin axes for supporting up to 2 lines with different y axes
#twin1 = ax.twinx()

#data_An = an_final.loc['2017-05-01 00:00':'2017-12-31 00:00'].resample('12H').mean()
data_GPP = df_Comb['GPP_f'].loc['2018-05-01 00:00':'2018-12-31 00:00'].resample('12H').mean()

#plot values
#p1 = ax.plot(data_An.index,data_An.values, color="b", label="A-gs Assimilation")
p1, = ax.plot(data_GPP.index,data_GPP.values, color="r", label="GPP_measured")

    

#ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
#twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])

        
    #twin1.invert_yaxis()
    #set axis labels
ax.set_xlabel("Time")
ax.set_ylabel("Assimilation [mg/m2/s]")
#ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='1M'))

fig.suptitle('Assimilation in 2018, resampled to 12 hours')
    
fig.autofmt_xdate() #allows you to auto-align the x axis labels

ax.legend()

plt.show()

#an_final.loc['2017-01-01 00:00':'2017-12-31 00:00'].resample('12H').mean().plot(title='Assimilation in 2017, resampled to 12 hours',ylabel='Assimilation [mg/m2/s]')

In [ ]:

fig,ax=plt.subplots() #put this inside the brackets constrained_layout=True
fig.subplots_adjust(right=0.75)
plt.rcParams["figure.figsize"] = (8,4)

#construct twin axes for supporting up to 2 lines with different y axes
#twin1 = ax.twinx()

data_An = an_final.loc['2018-05-01 00:00':'2018-12-31 00:00'].resample('12H').mean()
data_GPP = df_Comb['GPP_f'].loc['2018-05-01 00:00':'2018-12-31 00:00'].resample('12H').mean()

#plot values
p1 = ax.plot(data_An.index,data_An.values, color="b", label="A-gs Assimilation")
p2, = ax.plot(data_GPP.index,data_GPP.values, color="r", label="GPP_measured")

    

#ax.set_ylim(yaxis_min_maxes[0][0],yaxis_min_maxes[0][1])
#twin1.set_ylim(yaxis_min_maxes[1][0],yaxis_min_maxes[1][1])

        
    #twin1.invert_yaxis()
    #set axis labels
ax.set_xlabel("Time")
ax.set_ylabel("Assimilation [mg/m2/s]")
#ax.xaxis.set_ticks(pd.date_range(start=time_start, end=time_end, freq='1M'))

fig.suptitle('Assimilation in 2018, resampled to 12 hours')
    
fig.autofmt_xdate() #allows you to auto-align the x axis labels

ax.legend()

plt.show()

#an_final.loc['2017-01-01 00:00':'2017-12-31 00:00'].resample('12H').mean().plot(title='Assimilation in 2017, resampled to 12 hours',ylabel='Assimilation [mg/m2/s]')

In [ ]:
an_final

In [ ]:
df_EC.columns

In [ ]:
df_EC['Flx_Lo-CO2'].loc['2017-01-01 00:00':'2017-12-31 00:00'].plot()

In [ ]:
df_Comb.columns

In [ ]:
df_meteo.columns